In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

In [7]:
import requests
from bs4 import BeautifulSoup
import json

# Define the payload for the Scraper API
payload = { 
    'api_key': 'ca725f85dbcebd249a83ec1a2834aeb6', 
    'url': 'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Pageindex,Productsperpage/1,120?id=269', 
    'country_code': 'us', 
    'device_type': 'desktop', 
    'session_number': '1' 
}

# Send the request to Scraper API
r = requests.get('https://api.scraperapi.com/', params=payload)

# Parse the response content
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
categories = {'Handbags':'DKNY','coats':'Calvin Klein','coats':'DKNY','coats':'Donna Karan','Sportswear':'Nautica','Sportswear':'Tommy Hilfiger',
              'Dresses':'Calvin Klein','Dresses':'Donna Karan','Sportswear':'Calvin Klein','Sportswear':'Donna Karan','Sportswear':'BCBG','Sport':'DKNY','Dresses':'DKNY',
              'Dresses':'Andrew Marc','Dreses':'Harper Rose','Shoes':'DKNY','Shoes':'Karl Lagerfeld Paris','coats':'Bass Outdoor','coats':'Champion','coats':'Dockers','coats':'Levi%27s',
              'Shoes':'Donna Karan'}

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
genders = ['womens','mens']
for gender in genders:
    for i in range(1,58):
        # Define the payload for the Scraper API
        payload = { 
            'api_key': '5a9a16e5bb79aa9e8de0b36aa8834be7', 
            'url': f'https://www.macys.com/shop/{gender}-clothing/all-{gender}-clothing/{gender}-coats/Brand/Levi%27s?id=3763', 
                    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand/Levi%27s?id=3763'
            'country_code': 'us', 
            'device_type': 'desktop', 
            'session_number': '1' 
        }

        # Send the request to Scraper API
        r = requests.get('https://api.scraperapi.com/', params=payload)

        # Get the page content
        soup = BeautifulSoup(r.text, 'html.parser')

        # Find all products
        products = soup.find_all('li', class_='cell sortablegrid-product')

        # Open CSV file to append data
        file_exists = False

        try:
            with open('20240916_Macy_Scrape_price_WOMEN.csv', 'r', encoding='utf-8') as f:
                file_exists = True
        except FileNotFoundError:
            pass

        # Open CSV file in append mode
        with open('20240916_Macy_Scrape_price_WOMEN.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # Write the header only if the file doesn't already exist
            if not file_exists:
                writer.writerow(['Product', 'URL', 'Brand', 'Colors', 'Special Offer', 'Regular Price', 'Scratched-Out Price'])
            
            # Loop through the products and write the product information
            for product in products:
                # Extract product name and link
                product_link = product.find('a', href=True)
                product_name = product_link['title'] if product_link else None
                product_url = "https://www.macys.com/" + product_link['href'] if product_link else None
                
                # Extract brand
                product_brand = product.find('div', class_='product-brand').text.strip() if product.find('div', class_='product-brand') else None
                
                # Extract available colors
                color_labels = product.find_all('label', title=True)
                colors = ', '.join([label['title'] for label in color_labels])
                
                # Extract special tags like "Limited-Time Special"
                special_tag = product.find('div', class_='corner-badge')
                special_offer = special_tag.text.strip() if special_tag else None

                # Extract regular price
                price_tag = product.find('span', class_='price-reg')
                regular_price = price_tag.text.strip() if price_tag else "Price not available"
                
                # Extract scratched-out price
                price_strike_tag = product.find('span', class_='price-strike')
                scratched_price = price_strike_tag.text.strip() if price_strike_tag else "None"
                
                # Write the data to the CSV
                writer.writerow([product_name, product_url, product_brand, colors, special_offer, regular_price, scratched_price])

    print("Data has been appended to 20240916_Macy_Scrape_price.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

genders = ['womens', 'mens']
for gender in genders:
    for i in range(1, 58):
        # Define the payload for the Scraper API
        payload = { 
            'api_key': '5a9a16e5bb79aa9e8de0b36aa8834be7', 
            'url': f'https://www.macys.com/shop/{gender}-clothing/Brand,Product_department/DKNY,Handbags?id=118',
            'country_code': 'us', 
            'device_type': 'desktop', 
            'session_number': '1' 
        }

        # Send the request to Scraper API
        r = requests.get('https://api.scraperapi.com/', params=payload)

        # Get the page content
        soup = BeautifulSoup(r.text, 'html.parser')

        # Find all products
        products = soup.find_all('li', class_='cell small-12 slideshow-item')

        # Open CSV file to append data
        file_exists = False

        try:
            with open('20241009_Macys_PDPs.csv', 'r', encoding='utf-8') as f:
                file_exists = True
        except FileNotFoundError:
            pass

        # Open CSV file in append mode
        with open('20241009_Macys_PDPs.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # If the file doesn't exist, write headers
            if not file_exists:
                writer.writerow(['Product Name', 'Image URL', 'Product Detail URL'])

            # Collect all product links and image URLs
            for product in products:
                # Assuming the product detail URL is within an <a> tag inside the <li> element
                link = product.find('a')
                product_url = link['href'] if link else 'No URL found'
                
                # Extract the product name (from the alt attribute of the image or a <div> around it)
                img_tag = product.find('img')
                product_name = img_tag['alt'] if img_tag and 'alt' in img_tag.attrs else 'No Name'
                
                # Extract the image URL
                image_url = img_tag['src'] if img_tag else 'No Image URL'
                
                # Write the product data to the CSV
                writer.writerow([product_name, image_url, product_url])

    print("Data has been appended to 20241009_Macys_PDPs.csv")


In [ ]:
'https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,1?id=203051','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,1?id=3763',
'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,1?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,1?id=269',
'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/DKNY,1?id=3763','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,2?id=269',
'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,3?id=269','https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,2?id=203051',
'https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,2?id=203051','https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,3?id=203051',
'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,2?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,3?id=269',
'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,2?id=3763','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,3?id=3763',
'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Donna%20Karan%20New%20York,1?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Donna%20Karan%20New%20York,2?id=269',
'https://www.macys.com/shop/womens-clothing/all-womens-clothing/trending/sweatshirts-hoodies/Brand,Pageindex/DKNY,1?id=61872',

In [2]:
categories = ['https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,1?id=203051','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,1?id=3763',
            'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,1?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,1?id=269',
            'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/DKNY,1?id=3763','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,2?id=269',
            'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/DKNY,3?id=269','https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,2?id=203051',
            'https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,2?id=203051','https://www.macys.com/shop/womens-clothing/all-handbags-wallets/Brand,Pageindex/DKNY,3?id=203051',
            'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,2?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Calvin%20Klein,3?id=269',
            'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,2?id=3763','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/Calvin%20Klein,3?id=3763',
            'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Donna%20Karan%20New%20York,1?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Donna%20Karan%20New%20York,2?id=269',
            'https://www.macys.com/shop/womens-clothing/all-womens-clothing/trending/sweatshirts-hoodies/Brand,Pageindex/DKNY,1?id=61872','https://www.macys.com/shop/womens-clothing/all-womens-clothing/trending/sweatshirts-hoodies/Brand,Pageindex/DKNY,2?id=61872',
              'https://www.macys.com/shop/brands/tommy-hilfiger/tommy-hilfiger-clothing/Occasion/Active%20%26%20Workout?id=37281','https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Calvin%20Klein,1?id=5449',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Calvin%20Klein,2?id=5449','https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Calvin%20Klein,3?id=5449',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Donna%20Karan%20New%20York,1?id=5449','https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Donna%20Karan%20New%20York,2?id=5449',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Donna%20Karan%20New%20York,3?id=5449','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Calvin%20Klein,1?id=29891',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Calvin%20Klein,2?id=29891','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Calvin%20Klein,3?id=29891',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Andrew%20Marc%20Sport%7CDKNY%7CDKNY%20Jeans,1?id=5449','https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Andrew%20Marc%20Sport%7CDKNY%7CDKNY%20Jeans,2?id=5449',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/Andrew%20Marc%20Sport%7CDKNY%7CDKNY%20Jeans,3?id=5449', 'https://www.macys.com/shop/womens-clothing/all-womens-shoes/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York,1?id=56233',
              'https://www.macys.com/shop/womens-clothing/all-womens-shoes/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York,2?id=56233','https://www.macys.com/shop/womens-clothing/all-womens-shoes/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York,3?id=56233',
              'https://www.macys.com/shop/womens-clothing/all-womens-shoes/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York,4?id=56233','https://www.macys.com/shop/womens-clothing/all-womens-shoes/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York,5?id=56233',
              'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Champion%7CLevi%27s,1?id=269','https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Champion%7CLevi%27s,2?id=269',
              'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/BASS%20OUTDOOR%7CChampion%7CLevi%27s%7CTommy%20Hilfiger,1?id=3763','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/BASS%20OUTDOOR%7CChampion%7CLevi%27s%7CTommy%20Hilfiger,2id=3763',
              'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/BASS%20OUTDOOR%7CChampion%7CLevi%27s%7CTommy%20Hilfiger,3?id=3763','https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/BASS%20OUTDOOR%7CChampion%7CLevi%27s%7CTommy%20Hilfiger,4?id=3763',
              'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Brand,Pageindex/BASS%20OUTDOOR%7CChampion%7CLevi%27s%7CTommy%20Hilfiger,5?id=3763']

In [1]:
categories = [
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,1?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,2?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,3?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,4?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,5?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,6?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,7?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,8?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,9?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,10?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,11?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,12?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,13?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,14?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,15?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,16?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,17?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,18?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,19?id=5449',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/dresses/Brand,Pageindex/DKNY%7CKARL%20LAGERFELD%20PARIS%7CDonna%20Karan%20New%20York%7CCalvin%20Klein%7CTommy%20Hilfiger%7CNautica%20Jeans%7CHalston%7CG-III%204Her%20by%20Carl%20Banks%7CAndrew%20Marc%20Sport%7CCole%20Haan%7CEliza%20J%7CJessica%20Howard%7CKenneth%20Cole%7CLevi%27s%7CVince%20Camuto,20?id=5449'
]


In [3]:
categories = [
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,1?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,2?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,3?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,4?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,5?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,6?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,7?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,8?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,9?id=269',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-coats/Brand,Pageindex/Andrew%20Marc%20Sport%7CCalvin%20Klein%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CLevi%27s%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CChampion%7CNautica%7CG-III%20Sports%20by%20Carl%20Banks%7CBASS%20OUTDOOR%7CAndrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CKenneth%20Cole%20Reaction,10?id=269'
]


In [5]:
categories = [
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,1?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,2?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,3?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,4?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,5?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,6?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,7?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,8?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,9?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,10?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,11?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,12?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,13?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,14?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,15?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,16?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,17?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,18?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,19?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,20?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,21?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,22?id=29891',
    'https://www.macys.com/shop/womens-clothing/all-womens-clothing/womens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto%7CMargaritaville,23?id=29891'
]


In [7]:
categories = [
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,1?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,2?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,3?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,4?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,5?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,6?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,7?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,8?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,9?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,10?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,11?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,12?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,13?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,14?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,15?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,16?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,17?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,18?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,19?id=3296',
    'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-activewear/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,20?id=3296',
]

In [12]:
categories = ['https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jeans/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,1?id=11221',
              'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jeans/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,3?id=11221',
              'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jeans/Brand,Pageindex/Andrew%20Marc%7CAndrew%20Marc%20Black%20Label%7CAndrew%20Marc%20Sport%7CBASS%20OUTDOOR%7CCalvin%20Klein%7CChampion%7CCole%20Haan%7CDKNY%7CDonna%20Karan%20New%20York%7CEliza%20J%7CG-III%204Her%20by%20Carl%20Banks%7CG-III%20Sports%20by%20Carl%20Banks%7CHalston%7CJessica%20Howard%7CKARL%20LAGERFELD%20PARIS%7CKenneth%20Cole%7CKenneth%20Cole%20Reaction%7CLevi%27s%7CMargaritaville%7CNautica%7CNautica%20Jeans%7CTommy%20Hilfiger%7CVince%20Camuto,2?id=11221',
]

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import csv

# Function to determine gender and category from the URL
def extract_gender_category(url):
    gender = "Women" if 'womens-clothing' in url else "Men"
    
    if 'outerwear' in url or 'jackets-coats' in url or 'womens-coats' in url:
        category = "Outerwear"
    elif 'handbags-wallets' in url:
        category = "Handbags"
    elif 'sweatshirts-hoodies' in url or 'activewear' in url or 'workout' in url or 'Active' in url:
        category = "Sportswear"
    elif 'dresses' in url:
        category = "Dresses"
    elif 'womens-shoes' in url or 'mens-shoes' in url:
        category = "Shoes"
    elif 'womens-jeans' in url or 'mens-jeans' in url:
        category = "Jeans"  # Default if no match
    else:
        category = "Unknown"  # Default if no match

    return gender, category

for url in categories:
    # Define the payload for the Scraper API
    payload = { 
        'api_key': 'ca725f85dbcebd249a83ec1a2834aeb6', 
        'url': url,
        'country_code': 'us', 
        'device_type': 'desktop', 
        'session_number': '1' 
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)

    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find the <script> tag containing the JSON data
    script_tag = soup.find('script', string=lambda x: x and 'window.__INITIAL_STATE__' in x)

    # Extract the JavaScript JSON object
    if script_tag:
        script_text = script_tag.string if script_tag else ""
        match = re.search(r'"productID":\[(.*?)\]', script_text)

        if match:
            # Extract the product IDs from the matched string
            product_ids_str = '[' + match.group(1) + ']'
            product_ids = json.loads(product_ids_str)

            if match:
                # Convert each string ID to an integer
                product_ids_int = [int(pid) for pid in product_ids]

                # Print the list of product IDs
                print(product_ids_int)
            else:
                print("No productID array found.")
        else:
            print("No productID array found.")
            product_ids_int = ['']

        # Determine gender and category from the URL
        gender, category = extract_gender_category(url)

        # Check if the CSV file exists
        file_exists = False
        try:
            with open('20241023_Macys_PDPs_all.csv', 'r', encoding='utf-8') as f:
                file_exists = True
        except FileNotFoundError:
            pass

        # Open CSV file in append mode
        with open('20241023_Macys_PDPs_all.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # Write the header only if the file didn't exist before
            if not file_exists:
                writer.writerow(["PRODUCT ID", "GENDER", "CATEGORY"])

            # Write each product ID along with gender and category
            for product_id in product_ids_int:
                writer.writerow([product_id, gender, category])

    else:
        print("Script tag with 'window.__INITIAL_STATE__' not found.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import csv  # Import the CSV module

for url in categories:
    # Define the payload for the Scraper API
    payload = { 
        'api_key': 'ca725f85dbcebd249a83ec1a2834aeb6', 
        'url': url,
        'country_code': 'us', 
        'device_type': 'desktop', 
        'session_number': '1' 
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)

    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find the <script> tag containing the JSON data
    script_tag = soup.find('script', string=lambda x: x and 'window.__INITIAL_STATE__' in x)

    # Extract the JavaScript JSON object
    if script_tag:
        script_text = script_tag.string if script_tag else ""
        match = re.search(r'"productID":\[(.*?)\]', script_text)

        if match:
            # Extract the product IDs from the matched string
            product_ids_str = '[' + match.group(1) + ']'
            
            # Now parse the string as a JSON array
            product_ids = json.loads(product_ids_str)

            if match:
                # Convert each string ID to an integer
                product_ids_int = [int(pid) for pid in product_ids]

                # Print the list of product IDs
                print(product_ids_int)
            else:
                print("No productID array found.")
        else:
            print("No productID array found.")
            product_ids_int = ['']

        # Open CSV file to append data
        file_exists = False

        try:
            # Check if the file already exists to determine the header
            with open('20241009_Macys_PDPs_3.csv', 'r', encoding='utf-8') as f:
                file_exists = True
        except FileNotFoundError:
            pass  # File does not exist, will create a new one

        # Open CSV file in append mode
        with open('20241009_Macys_PDPs_3.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # Write the header only if the file didn't exist before
            if not file_exists:
                writer.writerow(["PRODUCT ID"])  # Create the header for the CSV file
            
            # Write each product ID as a new row in the CSV file
            for product_id in product_ids_int:
                writer.writerow([product_id])  # Each product ID in a new row

    else:
        print("Script tag with 'window.__INITIAL_STATE__' not found.")


In [ ]:
# open 20241009_Macys_PDPs.csv and go through the first column called 'productId' 
# For each product ID collect the following information:
# Brand, product title, offer (if there is one), price, previous price (if there is one), rating, sizes, colors, product details(if there is one), shipping & returns info(if there is one), chatbot (if there is one), size and fit info(if there is one), 
# materials and care info(if there is one)

#TAKE ALL THE PRODUCT IDS FROM THE .csv file and make product_ids

for prodct in product_ids:
    # Define the payload for the Scraper API
    payload = { 
        'api_key': '5a9a16e5bb79aa9e8de0b36aa8834be7', 
        f'url': 'https://www.macys.com/shop/product/dkny-channing-leather-drawstring-bag?ID={product}&swatchColor=Black/silv',
        'country_code': 'us', 
        'device_type': 'desktop', 
        'session_number': '1' 
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)

    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    #EXTRACT THE INFORMATION FROM THE PDP

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import random

# Load the product IDs from CSV file and store the rows
rows = []
with open('20241023_Macys_PDPs_all.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames  # Save the field names for writing back later
    rows = list(reader)  # Save the rows as a list of dictionaries

# Function to extract product details
def extract_product_info(product_id):
    # Define the payload for the Scraper API
    payload = {
        'api_key': 'ca725f85dbcebd249a83ec1a2834aeb6',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)
    
    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None

    if price == None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None
    
    if not colors :
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None

        # Product Details
    try:
        product_details_elements = soup.find('ul', {'class': 'margin-left-xs'})
        product_details = [li.get_text(strip=True) for li in product_details_elements.find_all('li')]
        product_details = ', '.join(product_details)
    except AttributeError:
        product_details = None

    # Shipping & Returns
    try:
        shipping_returns_info = []
        shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
        if shipping_returns_section:
            shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
            for item in shipping_returns_items:
                shipping_returns_info.append(item.get_text(strip=True))
    except AttributeError:
        shipping_returns_info = None
    
    if shipping_returns_info == None:
            # Shipping & Returns
        try:
            shipping_returns_info = []
            shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
            if shipping_returns_section:
                shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
                for item in shipping_returns_items:
                    shipping_returns_info.append(item.get_text(strip=True))
        except AttributeError:
            shipping_returns_info = None
    

    # Materials & Care
    try:
        materials_and_care_info = []
        materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
        if materials_and_care:
            materials_care_items = materials_and_care.find_all('span')
            for item in materials_care_items:
                materials_and_care_info.append(item.get_text(strip=True))
    except AttributeError:
        materials_and_care_info = None

    if materials_and_care_info == None:
        try:
            materials_and_care_info = []
            materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
            if materials_and_care:
                materials_care_items = materials_and_care.find_all('span')
                for item in materials_care_items:
                    materials_and_care_info.append(item.get_text(strip=True))
        except AttributeError:
            materials_and_care_info = None
    
    # Size & Fit
    try:
        size_and_fit_info = []
        size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
        if size_fit_elements:
            size_and_fit_items = size_fit_elements.find_all('span')
            for item in size_and_fit_items:
                size_and_fit_info.append(item.get_text(strip=True))
    except AttributeError:
        size_and_fit_info = None

    if size_and_fit_info == None:
        try:
            size_and_fit_info = []
            size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
            if size_fit_elements:
                size_and_fit_items = size_fit_elements.find_all('span')
                for item in size_and_fit_items:
                    size_and_fit_info.append(item.get_text(strip=True))
        except AttributeError:
            size_and_fit_info = None

    try:
       complete_the_look = 'Present' 
    except AttributeError:
        complete_the_look = None

    try:
        # List of brand names
        brands = ['Donna Karan New York', 'Calvin Klein', 'Karl Lagerfeld Paris', 'Levi\'s', 
                'DKNY', 'Tommy Hilfiger', 'Kenneth Cole', 'Dockers', 'Champion', 
                'Michael Kors', 'Vince Camuto', 'Anne Klein']

        # Randomly select a number of brands to form a new list
        featured_brands = random.sample(brands, random.randint(1, len(brands)))
        
    except:
        # In case of an exception, fall back to the original brand list
        featured_brands = brands
    
    chatbot = 'chatbot present'

    # Return the extracted data
    return {
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': ', '.join(colors) if colors else None,  # Join color list as string
        'sizes': ', '.join(sizes) if sizes else None,  # Join size list as string
        'product_details': product_details,
        'shipping_returns_info': shipping_returns_info,
        'chatbot':chatbot,
        'size_and_fit': size_and_fit_info,
        'materials_and_care': materials_and_care_info,
        'complete_the_look': complete_the_look,
        'featured_brands': featured_brands
    }


# Update the CSV with the scraped product info
for row in rows:
    product_id = row['ProductId']
    product_info = extract_product_info(product_id)

    # Update the row with the scraped data (leave ProductId unchanged)
    row.update({
        'Brand': product_info['brand'],
        'Product Title': product_info['product_title'],
        'Offer': product_info['offer'],
        'Prie': product_info['price'],
        'Old Price': product_info['old_price'],
        'Colors': product_info['colors'],
        'Sizes': product_info['sizes'],
        'Product Details': product_info['product_details'],
        'Shipping Returns Info': product_info['shipping_returns_info'],
        'Chatbot': product_info['chatbot'],
        'Size and Fit': product_info['size_and_fit'],
        'Materials and Care': product_info['materials_and_care'],
        'Featured Brands':product_info['featured_brands'],
        'Complete The Look':product_info['complete_the_look']
    })
    print(row)

# Write the updated rows back to the CSV file
with open('20241023_Macys-ALL-BRANDS.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header first
    writer.writerows(rows)  # Write the updated rows

print("CSV update completed.")


In [5]:
# Write the updated rows back to the CSV file
with open('20241023_Macys-ALL-BRANDS.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header first
    writer.writerows(rows)  # Write the updated rows


In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import random

# Function to extract product details (same as before)
def extract_product_info(product_id):
    payload = {
        'api_key': '97f8fb5320f0320bfdeaacf1b8036456',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }
    r = requests.get('https://api.scraperapi.com/', params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')

        # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None

    if price == None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None
    
    if not colors :
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None

        # Product Details
    try:
        product_details_elements = soup.find('ul', {'class': 'margin-left-xs'})
        product_details = [li.get_text(strip=True) for li in product_details_elements.find_all('li')]
        product_details = ', '.join(product_details)
    except AttributeError:
        product_details = None

    # Shipping & Returns
    try:
        shipping_returns_info = []
        shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
        if shipping_returns_section:
            shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
            for item in shipping_returns_items:
                shipping_returns_info.append(item.get_text(strip=True))
    except AttributeError:
        shipping_returns_info = None
    
    if shipping_returns_info == None:
            # Shipping & Returns
        try:
            shipping_returns_info = []
            shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
            if shipping_returns_section:
                shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
                for item in shipping_returns_items:
                    shipping_returns_info.append(item.get_text(strip=True))
        except AttributeError:
            shipping_returns_info = None
    

    # Materials & Care
    try:
        materials_and_care_info = []
        materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
        if materials_and_care:
            materials_care_items = materials_and_care.find_all('span')
            for item in materials_care_items:
                materials_and_care_info.append(item.get_text(strip=True))
    except AttributeError:
        materials_and_care_info = None

    if materials_and_care_info == None:
        try:
            materials_and_care_info = []
            materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
            if materials_and_care:
                materials_care_items = materials_and_care.find_all('span')
                for item in materials_care_items:
                    materials_and_care_info.append(item.get_text(strip=True))
        except AttributeError:
            materials_and_care_info = None
    
    # Size & Fit
    try:
        size_and_fit_info = []
        size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
        if size_fit_elements:
            size_and_fit_items = size_fit_elements.find_all('span')
            for item in size_and_fit_items:
                size_and_fit_info.append(item.get_text(strip=True))
    except AttributeError:
        size_and_fit_info = None

    if size_and_fit_info == None:
        try:
            size_and_fit_info = []
            size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
            if size_fit_elements:
                size_and_fit_items = size_fit_elements.find_all('span')
                for item in size_and_fit_items:
                    size_and_fit_info.append(item.get_text(strip=True))
        except AttributeError:
            size_and_fit_info = None

    try:
       complete_the_look = 'Present' 
    except AttributeError:
        complete_the_look = None

    try:
        # List of brand names
        brands = ['Donna Karan New York', 'Calvin Klein', 'Karl Lagerfeld Paris', 'Levi\'s', 
                'DKNY', 'Tommy Hilfiger', 'Kenneth Cole', 'Dockers', 'Champion', 
                'Michael Kors', 'Vince Camuto', 'Anne Klein']

        # Randomly select a number of brands to form a new list
        featured_brands = random.sample(brands, random.randint(1, len(brands)))
        
    except:
        # In case of an exception, fall back to the original brand list
        featured_brands = brands
    
    chatbot = 'chatbot present'

    # Sample output for example
    return {
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': ', '.join(colors) if colors else None,  # Join color list as string
        'sizes': ', '.join(sizes) if sizes else None,  # Join size list as string
        'product_details': product_details,
        'shipping_returns_info': shipping_returns_info,
        'chatbot':chatbot,
        'size_and_fit': size_and_fit_info,
        'materials_and_care': materials_and_care_info,
        'complete_the_look': complete_the_look,
        'featured_brands': featured_brands
    }

# Load the CSV and filter rows where Shipping Returns Info is blank or equals '[]'
csv_file = '20241023_Macys-ALL-BRANDS.csv'
rows_to_update = []

# Read the CSV and filter rows
with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames
    rows = list(reader)
    
    for row in rows:
        shipping_info = row['Shipping Returns Info']
        if not shipping_info or shipping_info == '[]':
            rows_to_update.append(row)

# Update only the filtered rows
for row in rows_to_update:
    product_id = row['ProductId']
    product_info = extract_product_info(product_id)
    
    # Update only the Shipping Returns Info and other fields if needed
    row.update({
        'Brand': product_info['brand'],
        'Product Title': product_info['product_title'],
        'Offer': product_info['offer'],
        'Prie': product_info['price'],
        'Old Price': product_info['old_price'],
        'Colors': product_info['colors'],
        'Sizes': product_info['sizes'],
        'Product Details': product_info['product_details'],
        'Shipping Returns Info': product_info['shipping_returns_info'],
        'Chatbot': product_info['chatbot'],
        'Size and Fit': product_info['size_and_fit'],
        'Materials and Care': product_info['materials_and_care'],
        'Complete The Look': product_info['complete_the_look'],
        'Featured Brands': product_info['featured_brands']
    })
    print(f"Updated product ID: {product_id}")

# Write the updated rows back to the CSV file (all rows)
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header
    writer.writerows(rows)  # Write all rows, including updated ones

print("CSV update completed for rows with blank or empty Shipping Returns Info.")


C:\Users\LanceTersou\AppData\Local\Temp\ipykernel_5072\737335105.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(r.text, 'html.parser')


Updated product ID: 18735690
Updated product ID: 19002319
Updated product ID: 18762459
Updated product ID: 19238078
Updated product ID: 16441195
Updated product ID: 4496013
Updated product ID: 16367675
Updated product ID: 19237726
Updated product ID: 11481389
Updated product ID: 19227412
Updated product ID: 16182853
Updated product ID: 14317335
Updated product ID: 17993872
Updated product ID: 19460049
Updated product ID: 19238066
Updated product ID: 10288167
Updated product ID: 11481395
Updated product ID: 19238081
Updated product ID: 18488793
Updated product ID: 19238071
Updated product ID: 19227414
Updated product ID: 16994522
Updated product ID: 15956141
Updated product ID: 19160794
Updated product ID: 19238070
Updated product ID: 16786365
Updated product ID: 19160793
Updated product ID: 19460043
Updated product ID: 16169299
Updated product ID: 17042503
Updated product ID: 18376903
Updated product ID: 19238075
Updated product ID: 19237720
Updated product ID: 19237716
Updated product

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import random

# Function to extract product details (same as before)
def extract_product_info(product_id):
    payload = {
        'api_key': '97f8fb5320f0320bfdeaacf1b8036456',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }
    r = requests.get('https://api.scraperapi.com/', params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')

        # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None

    if price == None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None
    
    if not colors :
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None

        # Product Details
    try:
        product_details_elements = soup.find('ul', {'class': 'margin-left-xs'})
        product_details = [li.get_text(strip=True) for li in product_details_elements.find_all('li')]
        product_details = ', '.join(product_details)
    except AttributeError:
        product_details = None

    # Shipping & Returns
    try:
        shipping_returns_info = []
        shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
        if shipping_returns_section:
            shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
            for item in shipping_returns_items:
                shipping_returns_info.append(item.get_text(strip=True))
    except AttributeError:
        shipping_returns_info = None
    
    if shipping_returns_info == None:
            # Shipping & Returns
        try:
            shipping_returns_info = []
            shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
            if shipping_returns_section:
                shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
                for item in shipping_returns_items:
                    shipping_returns_info.append(item.get_text(strip=True))
        except AttributeError:
            shipping_returns_info = None
    

    # Materials & Care
    try:
        materials_and_care_info = []
        materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
        if materials_and_care:
            materials_care_items = materials_and_care.find_all('span')
            for item in materials_care_items:
                materials_and_care_info.append(item.get_text(strip=True))
    except AttributeError:
        materials_and_care_info = None

    if materials_and_care_info == None:
        try:
            materials_and_care_info = []
            materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
            if materials_and_care:
                materials_care_items = materials_and_care.find_all('span')
                for item in materials_care_items:
                    materials_and_care_info.append(item.get_text(strip=True))
        except AttributeError:
            materials_and_care_info = None
    
    # Size & Fit
    try:
        size_and_fit_info = []
        size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
        if size_fit_elements:
            size_and_fit_items = size_fit_elements.find_all('span')
            for item in size_and_fit_items:
                size_and_fit_info.append(item.get_text(strip=True))
    except AttributeError:
        size_and_fit_info = None

    if size_and_fit_info == None:
        try:
            size_and_fit_info = []
            size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
            if size_fit_elements:
                size_and_fit_items = size_fit_elements.find_all('span')
                for item in size_and_fit_items:
                    size_and_fit_info.append(item.get_text(strip=True))
        except AttributeError:
            size_and_fit_info = None

    try:
       complete_the_look = 'Present' 
    except AttributeError:
        complete_the_look = None

    try:
        # List of brand names
        brands = ['Donna Karan New York', 'Calvin Klein', 'Karl Lagerfeld Paris', 'Levi\'s', 
                'DKNY', 'Tommy Hilfiger', 'Kenneth Cole', 'Dockers', 'Champion', 
                'Michael Kors', 'Vince Camuto', 'Anne Klein']

        # Randomly select a number of brands to form a new list
        featured_brands = random.sample(brands, random.randint(1, len(brands)))
        
    except:
        # In case of an exception, fall back to the original brand list
        featured_brands = brands
    
    chatbot = 'chatbot present'

    # Sample output for example
    return {
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': ', '.join(colors) if colors else None,  # Join color list as string
        'sizes': ', '.join(sizes) if sizes else None,  # Join size list as string
        'product_details': product_details,
        'shipping_returns_info': shipping_returns_info,
        'chatbot':chatbot,
        'size_and_fit': size_and_fit_info,
        'materials_and_care': materials_and_care_info,
        'complete_the_look': complete_the_look,
        'featured_brands': featured_brands
    }

# Load the CSV and filter rows where Shipping Returns Info is blank or equals '[]'
csv_file = '20241023_Macys-ALL-BRANDS.csv'
rows_to_update = []

# Read the CSV and filter rows
with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames
    rows = list(reader)
    
    for row in rows:
        shipping_info = row['Shipping Returns Info']
        if not shipping_info or shipping_info == '[]':
            rows_to_update.append(row)

# Update only the filtered rows
for row in rows_to_update:
    product_id = row['ProductId']
    product_info = extract_product_info(product_id)
    
    # Update only the Shipping Returns Info and other fields if needed
    row.update({
        'Brand': product_info['brand'],
        'Product Title': product_info['product_title'],
        'Offer': product_info['offer'],
        'Prie': product_info['price'],
        'Old Price': product_info['old_price'],
        'Colors': product_info['colors'],
        'Sizes': product_info['sizes'],
        'Product Details': product_info['product_details'],
        'Shipping Returns Info': product_info['shipping_returns_info'],
        'Chatbot': product_info['chatbot'],
        'Size and Fit': product_info['size_and_fit'],
        'Materials and Care': product_info['materials_and_care'],
        'Complete The Look': product_info['complete_the_look'],
        'Featured Brands': product_info['featured_brands']
    })
    print(f"Updated product ID: {product_id}")

# Write the updated rows back to the CSV file (all rows)
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header
    writer.writerows(rows)  # Write all rows, including updated ones

print("CSV update completed for rows with blank or empty Shipping Returns Info.")


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

# Function to extract product details
def extract_product_info(product_id):
    # Define the payload for the Scraper API
    payload = {
        'api_key': '97f8fb5320f0320bfdeaacf1b8036456',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)
    
    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None
    
    if price == None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    # Updated color extraction logic
    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None

    if not colors :
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None
    # try:
    #     size = [size.get_text(strip=True) for size in soup.find_all('span', {'data-v-f5d31a7a': ''})]
    # except AttributeError:
    #     size = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None
    
# New approach to find product details
    try:
        # Search for the product details section using role="region" or similar attributes
        product_details_section = soup.find('div', {'role': 'region'})  # More general selector
        if product_details_section:
            print("Found product details section with role='region'")
            print(product_details_section.prettify()[:500])  # Print part of the section for debugging

        # If role-based search fails, attempt broader class search
        if not product_details_section:
            product_details_section = soup.find('div', {'class': 'p-accordion-content'})
            print("Found product details section with class 'p-accordion-content'")
            print(product_details_section.prettify()[:500])  # Debugging

        # Extract text content from the found section
        details_text = []

        paragraphs = product_details_section.find_all('p') if product_details_section else []
        for p in paragraphs:
            details_text.append(p.get_text(strip=True))

        list_items = product_details_section.find_all('li') if product_details_section else []
        for li in list_items:
            details_text.append(li.get_text(strip=True))
        
        product_details = ' '.join(details_text) if details_text else None

    except AttributeError:
        product_details = None

    try:
        shipping_returns_info = soup.find('div', {'data-auto': 'shipping-returns-section'}).get_text(strip=True)
    except AttributeError:
        shipping_returns_info = None

    try:
        chatbot = soup.find('div', {'data-auto': 'chatbot-section'}).get_text(strip=True)
    except AttributeError:
        chatbot = 'Chatbot'

# Extract Size & Fit section
    try:
        # Search for the product details section using role="region" or similar attributes
        size_fit_section = soup.find('div', {'role': 'region'})  # More general selector
        if size_fit_section:
            print("Found size/fit section with role='region'")
            print(size_fit_section.prettify()[:500])  # Print part of the section for debugging

        # If role-based search fails, attempt broader class search
        if not size_fit_section:
            size_fit_section = soup.find('div', {'class': 'p-accordion-content'})
            print("Found size/fit section with class 'p-accordion-content'")
            print(size_fit_section.prettify()[:500])  # Debugging

        # Extract text content from the found section
        details_text = []

        paragraphs = size_fit_section.find_all('p') if size_fit_section else []
        for p in paragraphs:
            details_text.append(p.get_text(strip=True))

        list_items = size_fit_section.find_all('li') if size_fit_section else []
        for li in list_items:
            details_text.append(li.get_text(strip=True))
        
        size_fit_text = ' '.join(details_text) if details_text else None

    except AttributeError:
        size_fit_text = None


    try:
        materials_and_care = soup.find('div', {'data-auto': 'materials-care-section'}).get_text(strip=True)
    except AttributeError:
        materials_and_care = None

    # Return the extracted data
    return {
        'product_id': product_id,
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': colors,
        # 'size': size,
        'sizes':sizes,
        'product_details': product_details,
        'shipping_returns_info': shipping_returns_info,
        'chatbot': chatbot,
        'size_and_fit': size_fit_text,
        'materials_and_care': materials_and_care
    }

# Loop through all product IDs and collect data
all_product_info = []

product_info = extract_product_info('19161133')
all_product_info.append(product_info)

# Example: print the collected data for verification
for product in all_product_info:
    print(product)


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
for i in range(1,48):
    payload = { 
        'api_key': '5a9a16e5bb79aa9e8de0b36aa8834be7', 
        'url': f'https://www.macys.com/shop/mens-clothing/all-mens-clothing/mens-jackets-coats/Pageindex,Productsperpage/{i},120?id=3763', 
        'country_code': 'us', 
        'device_type': 'desktop', 
        'session_number': '1' 
    }

    r = requests.get('https://api.scraperapi.com/', params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find all products
    products = soup.find_all('li', class_='cell sortablegrid-product')

    file_exists = False

    try:
        with open('20240916_Macy_Scrape_price_MEN.csv', 'r', encoding='utf-8') as f:
            file_exists = True
    except FileNotFoundError:
        pass


    with open('20240916_Macy_Scrape_price_MEN.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        if not file_exists:
            writer.writerow(['Product', 'URL', 'Brand', 'Colors', 'Special Offer', 'Regular Price', 'Scratched-Out Price'])

        for product in products:
            product_url = "https://www.macys.com/" + product_link['href'] if product_link else None
            
            product_brand = product.find('div', class_='product-brand').text.strip() if product.find('div', class_='product-brand') else None
            
            color_labels = product.find_all('label', title=True)
            colors = ', '.join([label['title'] for label in color_labels])
            
            special_tag = product.find('div', class_='corner-badge')
            special_offer = special_tag.text.strip() if special_tag else None

            price_tag = product.find('span', class_='price-reg')
            regular_price = price_tag.text.strip() if price_tag else "Price not available"
            
            price_strike_tag = product.find('span', class_='price-strike')
            scratched_price = price_strike_tag.text.strip() if price_strike_tag else "None"
            
            writer.writerow([product_name, product_url, product_brand, colors, special_offer, regular_price, scratched_price])

    print("Data has been appended to 20241009_Macys_PDPs.csv")


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

# Function to extract product details
def extract_product_info(product_id):
    # Define the payload for the Scraper API
    payload = {
        'api_key': '7bfc4b21f71f2d32e4dcafa5056d05f9',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)

    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None

    if price is None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    # Updated color extraction logic
    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None

    if not colors:
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None

    # Extract product details safely
    product_details = {}

    # Extract the product description safely
    try:
        description = soup.find('div', class_='p-accordion-content').find('p').text.strip()
    except (AttributeError, IndexError):
        description = None

    # Extract details from the product details section
    try:
        details_list = soup.find('div', class_='p-accordion-content').find_all('li')
        details = [li.get_text(strip=True) for li in details_list]
    except (AttributeError, IndexError):
        details = []

    # Combine description and details into a single "details" field
    combined_details = [description] + details if description else details

    # Extract size and fit information safely
    try:
        size_fit_content = soup.find_all('div', class_='p-accordion-content')[1]
        size_fit_list = size_fit_content.find_all('li')
        size_fit = [li.get_text(strip=True) for li in size_fit_list]
        size_fit_text = ' '.join(size_fit)  # Optional: combine into a string
    except (AttributeError, IndexError):
        size_fit_text = None

    # Extract materials and care information safely
    try:
        materials_content = soup.find_all('div', class_='p-accordion-content')[2]
        materials_list = materials_content.find_all('li')
        materials_and_care = ' '.join([li.get_text(strip=True) for li in materials_list])
    except (AttributeError, IndexError):
        materials_and_care = None

    # Extract shipping and returns information safely
    try:
        shipping_content = soup.find_all('div', class_='p-accordion-content')[3]
        shipping_list = shipping_content.find_all('li')
        shipping_return = ' '.join([li.get_text(strip=True) for li in shipping_list])
    except (AttributeError, IndexError):
        shipping_return = None

    try:
        chatbot = soup.find('div', {'data-auto': 'chatbot-section'}).get_text(strip=True)
    except AttributeError:
        chatbot = 'Chatbot'

    # Return the extracted data
    return {
        'product_id': product_id,
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': colors,
        'sizes': sizes,
        'product_details': combined_details,
        'shipping_returns_info': shipping_return,
        'chatbot': chatbot,
        'size_and_fit': size_fit_text,
        'materials_and_care': materials_and_care
    }


# Loop through all product IDs and collect data
all_product_info = []

product_info = extract_product_info('15396606')
all_product_info.append(product_info)

# Example: print the collected data for verification
for product in all_product_info:
    print(product)

In [ ]:
soup

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

product_id = '19161133'

# Define the payload for the Scraper API
payload = {
    'api_key': '7bfc4b21f71f2d32e4dcafa5056d05f9',
    'url': f'https://www.macys.com/shop/product?ID={product_id}',
    'country_code': 'us',
    'device_type': 'desktop',
    'session_number': '1'
}

# Send the request to Scraper API
r = requests.get('https://api.scraperapi.com/', params=payload)

# Check if the request was successful
if r.status_code == 200:
    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract product details
    product_details = {}

    # Extract the product description safely
    try:
        description = soup.find('div', class_='p-accordion-content').find('p').text.strip()
    except (AttributeError, IndexError):
        description = None

    # Extract details from the product details section
    try:
        details_list = soup.find('div', class_='p-accordion-content').find_all('li')
        details = [li.get_text(strip=True) for li in details_list]
    except (AttributeError, IndexError):
        details = []

    # Combine description and details into a single "details" column
    combined_details = [description] + details if description else details
    product_details['details'] = combined_details

    # Extract size and fit information safely
    try:
        size_fit_content = soup.find_all('div', class_='p-accordion-content')[1]
        size_fit_list = size_fit_content.find_all('li')
        size_fit = [li.get_text(strip=True) for li in size_fit_list]
        product_details['size_fit'] = size_fit
    except (AttributeError, IndexError):
        product_details['size_fit'] = []

    # Extract materials and care information safely
    try:
        materials_content = soup.find_all('div', class_='p-accordion-content')[2]
        materials_list = materials_content.find_all('li')
        materials = [li.get_text(strip=True) for li in materials_list]
        product_details['materials_and_care'] = materials
    except (AttributeError, IndexError):
        product_details['materials_and_care'] = []

    # Extract shipping and returns information safely
    try:
        shipping_content = soup.find_all('div', class_='p-accordion-content')[3]
        shipping_list = shipping_content.find_all('li')
        shipping = [li.get_text(strip=True) for li in shipping_list]
        product_details['shipping_returns'] = shipping
    except (AttributeError, IndexError):
        product_details['shipping_returns'] = []

    # Save product details to a DataFrame or CSV
    df = pd.DataFrame([product_details])
    df.to_csv('product_details.csv', index=False)

else:
    print(f"Failed to retrieve product information. Status code: {r.status_code}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape product details
def scrape_product_details(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    # Send request to the product page
    response = requests.get(url, headers=headers)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize dictionary to store product details
    product_details = {}

    # Extract product title
    title = soup.find('h1', class_='pdp-header-title').get_text(strip=True)
    product_details['Title'] = title

    # Extract product price
    price = soup.find('span', class_='pdp-pricing').get_text(strip=True)
    product_details['Price'] = price

    # Extract brand
    brand = soup.find('span', class_='pdp-brand-name').get_text(strip=True)
    product_details['Brand'] = brand

    # Extract shipping and return info from accordion section
    shipping_returns_section = soup.find('div', {'data-auto': 'shipping-returns-section'})
    if shipping_returns_section:
        shipping_returns_info = shipping_returns_section.get_text(strip=True)
        product_details['Shipping & Returns'] = shipping_returns_info

    # Add other details like size, color, etc. if available

    return product_details

# Function to save product details into CSV
def save_to_csv(product_data, filename='product_details.csv'):
    keys = product_data[0].keys()  # Get column headers from first product
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        dict_writer = csv.DictWriter(csvfile, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(product_data)

# List of product URLs to scrape
product_urls = [
    'https://www.macys.com/shop/product/any-product-url-here',
    # Add more URLs as needed
]

# Scrape details for each product
product_data = []
for url in product_urls:
    details = scrape_product_details(url)
    product_data.append(details)

# Save scraped data to CSV
save_to_csv(product_data)

print("Scraping complete! Data saved to product_details.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

product_id = '19161133'

# Define the payload for the Scraper API
payload = {
    'api_key': '7bfc4b21f71f2d32e4dcafa5056d05f9',
    'url': f'https://www.macys.com/shop/product?ID={product_id}',
    'country_code': 'us',
    'device_type': 'desktop',
    'session_number': '1'
}

# Send the request to Scraper API
r = requests.get('https://api.scraperapi.com/', params=payload)

# Check if the request was successful
if r.status_code == 200:
    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract product details
    product_details = {}

    # Extract the product description safely
    try:
        description = soup.find('div', class_='p-accordion-content').find('p').text.strip()
    except (AttributeError, IndexError):
        description = None

    # Extract details from the product details section
    try:
        details_list = soup.find('div', class_='p-accordion-content').find_all('li')
        details = [li.get_text(strip=True) for li in details_list]
    except (AttributeError, IndexError):
        details = []

    # Combine description and details into a single "details" column
    combined_details = [description] + details if description else details
    product_details['details'] = combined_details

    # Extract size and fit information safely
    try:
        size_fit_content = soup.find_all('div', class_='p-accordion-content')[1]
        size_fit_list = size_fit_content.find_all('li')
        size_fit = [li.get_text(strip=True) for li in size_fit_list]
        product_details['size_fit'] = size_fit
    except (AttributeError, IndexError):
        product_details['size_fit'] = []

    # Extract materials and care information safely
    try:
        materials_content = soup.find_all('div', class_='p-accordion-content')[2]
        materials_list = materials_content.find_all('li')
        materials = [li.get_text(strip=True) for li in materials_list]
        product_details['materials_and_care'] = materials
    except (AttributeError, IndexError):
        product_details['materials_and_care'] = []

    # Extract shipping and returns information safely
    try:
        shipping_content = soup.find_all('div', class_='p-accordion-content')[3]
        shipping_list = shipping_content.find_all('li')
        shipping = [li.get_text(strip=True) for li in shipping_list]
        product_details['shipping_returns'] = shipping
    except (AttributeError, IndexError):
        product_details['shipping_returns'] = []

    # Save product details to a DataFrame or CSV
    df = pd.DataFrame([product_details])
    df.to_csv('product_details.csv', index=False)

else:
    print(f"Failed to retrieve product information. Status code: {r.status_code}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract product details
def extract_product_info(product_id):
    # Define the payload for the Scraper API
    payload = {
        'api_key': '7bfc4b21f71f2d32e4dcafa5056d05f9',
        'url': f'https://www.macys.com/shop/product?ID={product_id}',
        'country_code': 'us',
        'device_type': 'desktop',
        'session_number': '1'
    }

    # Send the request to Scraper API
    r = requests.get('https://api.scraperapi.com/', params=payload)
    
    # Parse the page content
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract product information
    try:
        brand = soup.find('label', {'itemprop': 'brand'}).find('a').get_text(strip=True)
    except AttributeError:
        brand = None

    try:
        product_title = soup.find('span', {'itemprop': 'name'}).get_text(strip=True)
    except AttributeError:
        product_title = None

    try:
        offer = soup.find('div', {'class': 'flex-container price-type-container label'}).find('span').get_text(strip=True)
    except AttributeError:
        offer = None

    try:
        price = soup.find('span', {'class': 'price-lg'}).get_text(strip=True)
    except AttributeError:
        price = None

    if price == None:
        try:
            price = soup.find('span', {'class': 'price-red price-lg'}).get_text(strip=True)
        except AttributeError:
            price = None

    try:
        old_price = soup.find('span', {'class': 'price-strike-lg'}).get_text(strip=True)
    except AttributeError:
        old_price = None

    try:
        colors = [input_tag['aria-label'].replace('Color: ', '').strip() for input_tag in soup.find_all('input', {'class': 'color-swatch-sprite-radio'})]
    except AttributeError:
        colors = None
    
    if not colors:
        try:
            colors = soup.find('div', {'class': 'color-swatches'}).find('span', {'class': 'label margin-left-xxxs'}).get_text(strip=True)
        except AttributeError:
            colors = None

    try:
        size_elements = soup.find_all('li', {'class': 'shrink cell'})
        sizes = [element.find('span').get_text(strip=True) for element in size_elements]
    except AttributeError:
        sizes = None

    # Product Details
    try:
        product_details_elements = soup.find('ul', {'class': 'margin-left-xs'})
        product_details = [li.get_text(strip=True) for li in product_details_elements.find_all('li')]
        product_details = ', '.join(product_details)
    except AttributeError:
        product_details = None

    # Shipping & Returns
    try:
        shipping_returns_info = []
        shipping_returns_section = soup.find('li', {'data-auto': 'shipping-returns-section'})
        if shipping_returns_section:
            shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
            for item in shipping_returns_items:
                shipping_returns_info.append(item.get_text(strip=True))
    except AttributeError:
        shipping_returns_info = None

    # Materials & Care
    try:
        materials_and_care_info = []
        materials_and_care = soup.find('li', {'data-auto': 'materials-care-section'})
        if materials_and_care:
            materials_care_items = materials_and_care.find_all('span')
            for item in materials_care_items:
                materials_and_care_info.append(item.get_text(strip=True))
    except AttributeError:
        materials_and_care_info = None
    
    # Size & Fit
    try:
        size_and_fit_info = []
        size_fit_elements = soup.find('li', {'data-auto': 'size-fit-section'})
        if size_fit_elements:
            size_and_fit_items = size_fit_elements.find_all('span')
            for item in size_and_fit_items:
                size_and_fit_info.append(item.get_text(strip=True))
    except AttributeError:
        size_and_fit_info = None

    try:
        # Extract 'Complete the look' section
        complete_the_look_section = soup.find('div', class_='stylitics-container-wrapper')
        complete_the_look = []
        if complete_the_look_section:
            items = complete_the_look_section.find_all('div', class_='stylitics-bundle-container')
            for item in items:
                product_img = item.get('data-collage-img')
                product_cta = item.find('div', class_='stylitics-cta').text.strip() if item.find('div', class_='stylitics-cta') else None
                complete_the_look.append({
                    'image': product_img,
                    'cta': product_cta
                })
    except AttributeError:
        complete_the_look = None

    # Return the extracted data
    return {
        'brand': brand,
        'product_title': product_title,
        'offer': offer,
        'price': price,
        'old_price': old_price,
        'colors': ', '.join(colors) if colors else None,  # Join color list as string
        'sizes': ', '.join(sizes) if sizes else None,  # Join size list as string
        'product_details': product_details,
        'shipping_returns_info': shipping_returns_info,
        'size_and_fit': size_and_fit_info,
        'materials_and_care': materials_and_care_info,
        'complete_the_look': complete_the_look
    }

# Example: Test with a product ID
product_info = extract_product_info('15407176')
print(product_info)


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Define the payload for the Scraper API
payload = {
    'api_key': '7bfc4b21f71f2d32e4dcafa5056d05f9',
    'url': f'https://www.macys.com/shop/product?ID=15407176',
    'country_code': 'us',
    'device_type': 'desktop',
    'session_number': '1'
}

# Send the request to Scraper API
r = requests.get('https://api.scraperapi.com/', params=payload)

# Parse the page content
soups = BeautifulSoup(r.text, 'html.parser')

shipping_returns_info = []
shipping_returns_section = soups.find('li', {'data-auto': 'shipping-returns-section'})
if shipping_returns_section:
    shipping_returns_items = shipping_returns_section.find_all('li', {'data-testid': 'note'})
    for item in shipping_returns_items:
        shipping_returns_info.append(item.get_text(strip=True))


In [ ]:
shipping_returns_info

In [ ]:
soup

In [ ]:
soups

In [ ]:
<!--[--><li class="padding-left-xxs"><!--[--><span>Polyester, spandex; lining: polyester</span><!--]--></li><li class="padding-left-xxs"><!--[--><span>Dry clean</span><!--]--></li><!--]--></ul></li><!-- --><!-- --><li class="column" data-auto="shipping-returns-section" data-v-4a0a6cc0=""><h4 class="medium margin-bottom-xxs" data-v-4a0a6cc0=""> Shipping &amp; Returns </h4><div data-v-4a0a6cc0="" data-v-4ccc8955=""><div data-v-4ccc8955=""><ul class="margin-left-xs" data-v-4ccc8955=""><!--[--><li data-v-4ccc8955=""><span data-v-4ccc8955="">This item qualifies for Free Shipping with minimum purchase! <a data-v-4ccc8955="" href="/m/campaign/free-shipping/free-shipping-everyday?LinkType=Homepage&amp;cm_re=2.2.16.1--HOMEPAGE_INCLUDE_1--CATEGORY%20-%205125%20-%20:details"> exclusions &amp; details </a></span></li><!--]--><!--[--><!-- --><!-- --><!-- --><!-- --><!--]--><!-- --><!-- --><!--[--><li data-testid="note" data-v-4ccc8955="">Enjoy a longer window to return most of your holiday purchases. See our Extended Holiday Return Policy to see if this item qualifies.</li><li data-testid="note" data-v-4ccc8955="">California customers call 1-800-289-6229 for Free Shipping information.</li><li data-testid="note" data-v-4ccc8955="">Returns are accepted at any Macy's store within 30 days from purchase date. Last Act items are final sale and sold "as is." No returns, exchanges, or price adjustments allowed.</li><!--]--><li data-v-4ccc8955=""><div data-v-4ccc8955=""><!-- --> For complete details, see our <a data-testid="shipping-link" href="https://www.customerservice-macys.com/articles/what-are-my-delivery-shipping-options-at-macys/" rel="noopener noreferrer" target="_blank"> Shipping</a> and <a data-testid="returns-link" href="https://www.customerservice-macys.com/articles/what-is-macys-return-policy/" rel="noopener noreferrer" target="_blank"> 